In [7]:
import os
os.chdir('/Users/alexascunceparis/Desktop/BSC/TCRranker/src')

# TRAINING THE MODEL TCR-p

In [9]:
import pandas as pd
from potential_calc import calculate_potential
from utils import parse_general_file, extract_specific_sequences
from extract_contacts import filter_contacts
from config import STRUCTURES_ANNOTATION_DIR, MODELS_DIR, PDB_DIR, DATA_DIR

## TRAIN MODEL FOR TCR-P BY POSITION (P1_P9)

### Define the folds

In [ ]:
import copy

def create_empty_contact_model(length):
    """Creates an empty contact model for a given peptide length."""
    return {f'P{i+1}': pd.DataFrame() for i in range(length)}

def main():
    # Step 1: List of PDB IDs to exclude from training
    validation_set = []
    training_set = []
    type_model = "9+all"

    # Dictionary to store models by peptide length
    contact_models = {}

    # Parse chain information from the general.txt file
    general_path = os.path.join(STRUCTURES_ANNOTATION_DIR, "general.txt")
    chain_dict = parse_general_file(general_path)

    # Path to the folder containing contact maps
    folder_path = os.path.join(DATA_DIR, "contact_maps")
    contact_files = [
        f for f in os.listdir(folder_path)
        if f.endswith('_contacts.csv')]
    
    print(len(contact_files), "contact files found in the folder.")

    # Step 4: Process each contact file
    if type_model == "pure":
        contact_maps_11 = []
        for contact_file in contact_files:
            pdb_id = contact_file.split('_')[0]
            pdb_path = os.path.join(PDB_DIR, f"{pdb_id}.pdb")
            _, _, peptide_sequence = extract_specific_sequences(pdb_path, chain_dict)
            peptide_length = len(peptide_sequence)
            if peptide_length <= 10:
                print(f"Processing contacts for contact map: {pdb_id}")
                
                # Read the contacts for the current PDB ID
                contacts = pd.read_csv(os.path.join(folder_path, contact_file))

                # Check if the pdb_id exists in chain_dict, if not, use the default settings
                chains = chain_dict.get(pdb_id, {
                    'tcra_chain': 'D',
                    'tcrb_chain': 'E',
                    'peptide_chain': 'C',
                    'b2m_chain': 'B',
                    'mhc_chain': 'A'})
                
                chain_dict[pdb_id] = chains

                if chains and all(chains.values()):
                    try:
                        # Filter contacts for TCR-peptide and TCR-MHC
                        contacts_TCR_p, contacts_TCR_MHC = filter_contacts(
                            contacts,
                            chains['tcra_chain'],
                            chains['tcrb_chain'],
                            chains['peptide_chain'],
                            chains['mhc_chain'])

                        if not contacts_TCR_p.empty:
                            if peptide_length not in contact_models:
                                contact_models[peptide_length] = create_empty_contact_model(peptide_length)

                            # Distribute contacts into appropriate positions (P1, P2, ..., Pn)
                            for _, contact in contacts_TCR_p.iterrows():
                                resid_to = contact['resid_to']

                                # Ensure resid_to falls within the peptide length
                                if 1 <= resid_to <= peptide_length:
                                    position = f'P{resid_to}'
                                    contact_models[peptide_length][position] = pd.concat([contact_models[peptide_length][position], contact.to_frame().T])
                    except Exception as e:
                        print(f"Error processing contact map {pdb_id}: {e}")
            else:
                contact_maps_11.append(contact_file)
        
        for contact_file in contact_maps_11:
            assignments_dict = {11: {1: 1, 2: 2, 3: 3, 4: 4, 5: 4, 6: 5, 7: 5, 8: 8, 9: 5, 10: 9, 11: 10},
                                12: {1: 1, 2: 2, 3: 3, 4: 2, 5: 5, 6: 7, 7: 7, 8: 6, 9: 9, 10: 9, 11: 9, 12: 10},
                                13: {1: 1, 2: 2, 3: 3, 4: 6, 5: 3, 6: 4, 7: 8, 8: 6, 9: 8, 10: 9, 11: 9, 12: 9, 13: 10}}
            
            pdb_id = contact_file.split('_')[0]
            pdb_path = os.path.join(PDB_DIR, f"{pdb_id}.pdb")
            _, _, peptide_sequence = extract_specific_sequences(pdb_path, chain_dict)
            peptide_length = len(peptide_sequence)

            print(f"Processing contacts for tcr with > 10 epitope length: {pdb_id}")
            contacts = pd.read_csv(os.path.join(folder_path, contact_file))
            chains = chain_dict.get(pdb_id, {
                'tcra_chain': 'D',
                'tcrb_chain': 'E',
                'peptide_chain': 'C',
                'b2m_chain': 'B',
                'mhc_chain': 'A'})
            chain_dict[pdb_id] = chains

            if chains and all(chains.values()):
                try:
                    contacts_TCR_p, contacts_TCR_MHC = filter_contacts(
                        contacts,
                        chains['tcra_chain'],
                        chains['tcrb_chain'],
                        chains['peptide_chain'],
                        chains['mhc_chain'])
                    if not contacts_TCR_p.empty:
                        if 11 not in contact_models:
                            contact_models[11] = copy.deepcopy(contact_models.get(10))
                        for _, contact in contacts_TCR_p.iterrows():
                            original_position = contact['resid_to']
                            resid_to = assignments_dict[peptide_length][original_position]
                            position = f'P{resid_to}'
                            contact_models[11][position] = pd.concat([contact_models[11][position], contact.to_frame().T])
                            
                except Exception as e:
                    print(f"Error processing contact map {pdb_id}: {e}")

    if type_model == "9+all":
        for contact_file in contact_files:
            pdb_id = contact_file.split('_')[0]
            pdb_path = os.path.join(PDB_DIR, f"{pdb_id}.pdb")
            _, _, peptide_sequence = extract_specific_sequences(pdb_path, chain_dict)
            peptide_length = len(peptide_sequence)
            assignments_dict = {
                4: {1: 1, 2: 2, 3: 3, 4: 9},
                8: {1: 1, 2: 2, 3: 3, 4: 5, 5: 5, 6: 7, 7: 8, 8: 9},
                9: {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9},
                10: {1: 1, 2: 2, 3: 3, 4: 4, 5: 6, 6: 5, 7: 6, 8: 6, 9: 8, 10: 9},
                11: {1: 1, 2: 2, 3: 3, 4: 4, 5: 4, 6: 4, 7: 8, 8: 8, 9: 7, 10: 8, 11: 9},
                12: {1: 1, 2: 2, 3: 3, 4: 2, 5: 5, 6: 7, 7: 7, 8: 6, 9: 9, 10: 9, 11: 9, 12: 9},
                13: {1: 1, 2: 2, 3: 3, 4: 6, 5: 3, 6: 4, 7: 8, 8: 6, 9: 8, 10: 9, 11: 9, 12: 9, 13: 9}}
            
            print(f"Processing contacts for contact map: {pdb_id}")    

            # Read the contacts for the current PDB ID
            contacts = pd.read_csv(os.path.join(folder_path, contact_file))

            # Check if the pdb_id exists in chain_dict, if not, use the default settings
            chains = chain_dict.get(pdb_id, {
                    'tcra_chain': 'D',
                    'tcrb_chain': 'E',
                    'peptide_chain': 'C',
                    'b2m_chain': 'B',
                    'mhc_chain': 'A'})
                
            chain_dict[pdb_id] = chains

            if chains and all(chains.values()):
                try:
                    # Filter contacts for TCR-peptide and TCR-MHC
                    contacts_TCR_p, contacts_TCR_MHC = filter_contacts(
                            contacts,
                            chains['tcra_chain'],
                            chains['tcrb_chain'],
                            chains['peptide_chain'],
                            chains['mhc_chain'])

                    if not contacts_TCR_p.empty:
                        if 9 not in contact_models:
                            contact_models[9] = create_empty_contact_model(9)

                        # Distribute contacts into appropriate positions (P1, P2, ..., Pn)
                        for _, contact in contacts_TCR_p.iterrows():
                            original_position = contact['resid_to']
                            resid_to = assignments_dict[peptide_length][original_position]

                            # Ensure resid_to falls within the peptide length
                            if 1 <= resid_to <= 9:
                                position = f'P{resid_to}'
                                contact_models[9][position] = pd.concat([contact_models[9][position], contact.to_frame().T])

                except Exception as e:
                    print(f"Error processing contact map {pdb_id}: {e}")

    # Step 5: Create directories and save potentials for each peptide length
    for length, model in contact_models.items():
        # Create a directory for the current peptide length if it doesn't exist
        try:
            length_directory = os.path.join(MODELS_DIR, f"TCR-p-L{length}")
            os.makedirs(length_directory, exist_ok=True)

            for position, contacts in model.items():
                if not contacts.empty:
                    print(f"Calculating TCR-peptide potential for length {length}, position {position}")

                    # Calculate the TCR-peptide potential for the current contact data
                    data_TCR_p = calculate_potential(contacts, peptide=True)
                    
                    # Generate the output file name based on the residue position
                    output_p = os.path.join(length_directory, f"tcr_p_potential_{position}.csv")
                    
                    # Save the result to a CSV file in the corresponding folder
                    data_TCR_p.to_csv(output_p, index=False)
                    print(f"TCR-peptide potential for length {length}, position {position} saved to {output_p}")
                else:
                    print(f"No valid TCR-peptide contacts for length {length}, position {position}.")
        except Exception as e:
            print(f"Error processing length {length}: {e}")
            
    print("Training complete for all contact maps.")

if __name__ == "__main__":
    main()

252 contact files found in the folder.
Processing contacts for PDB ID: 7n2p
Processing contacts for PDB ID: 8rlv
Processing contacts for PDB ID: 4jfd
Processing contacts for PDB ID: 7l1d
Processing contacts for PDB ID: 2gj6
Processing contacts for PDB ID: 5wkf
Processing contacts for PDB ID: 2ckb
Processing contacts for PDB ID: 7jwj
Processing contacts for PDB ID: 8gvb
Processing contacts for PDB ID: 8ryn
Processing contacts for PDB ID: 5ivx
Processing contacts for PDB ID: 5isz
Processing contacts for PDB ID: 7n4k
Processing contacts for PDB ID: 6bj3
Processing contacts for PDB ID: 1mwa
Processing contacts for PDB ID: 5c0b
Processing contacts for PDB ID: 5c08
Processing contacts for PDB ID: 3e3q
Processing contacts for PDB ID: 5wlg
Processing contacts for PDB ID: 7n2o
Processing contacts for PDB ID: 5jhd
Processing contacts for PDB ID: 6vm7
Processing contacts for PDB ID: 8v4z
Processing contacts for PDB ID: 8ryq
Processing contacts for PDB ID: 6zkz
Processing contacts for PDB ID: 4ftv

# TRAIN MODEL FOR TCR-MHC

### Select training set

In [ ]:
import os
import pandas as pd

def main():
    # Step 1: Create a list of PDB IDs to exclude from training
    exclude_pdb_ids = []
    selected_pdb_files = ['5brz', '5c0a', '6p64', '3gsn', '5jzi', '7l1d', '7pb2', '7nme', '6avg', '3ffc', '7n2p', '7n2q', '8enh', '4prh', '6mtm', '7dzn', '3kps', '4mji', '7r80', '8f5a', '7dzm', '8shi', '6uon', '8rlv', '5w1v', '7ndu', '7na5', '5ivx', '1g6r', '1mwa', '2ol3', '4mxq']

    # Step 2: Read chain information from the general.txt file
    chain_dict = parse_general_file('./structures_annotation/general.txt')

    # Step 3: Path to the folder containing contact maps
    folder_path = './contact_maps'
    contact_files = [
        f for f in os.listdir(folder_path)
        if f.endswith('_contacts.csv') and f.split('_')[0] in selected_pdb_files]
    print(len(contact_files), "contact files found in the folder.")

    # Initialize DataFrame to store all contacts
    all_contacts_TCR_MHC = pd.DataFrame()

    # Step 4: Iterate over all contact files
    for contact_file in contact_files:
        pdb_id = contact_file.split('_')[0]  # Extract the PDB ID from the file name
        
        if pdb_id in exclude_pdb_ids:  # Skip if the PDB ID is in the exclusion list
            print(f"Skipping PDB ID: {pdb_id} (in exclusion list)")
            continue

        print(f"Processing contacts for PDB ID: {pdb_id}")
        
        # Read the contacts for the current PDB ID
        contacts = pd.read_csv(os.path.join(folder_path, contact_file))
        chains = chain_dict.get(pdb_id, {
            'tcra_chain': 'D',
            'tcrb_chain': 'E',
            'peptide_chain': 'C',
            'b2m_chain': 'B',
            'mhc_chain': 'A'
        })
        chain_dict[pdb_id] = chains

        if chains and all(chains.values()):  # Ensure all chains are present
            try:
                # Filter contacts for the current PDB ID
                contacts_TCR_p, contacts_TCR_MHC = filter_contacts(
                    contacts,
                    chains['tcra_chain'],
                    chains['tcrb_chain'],
                    chains['peptide_chain'],
                    chains['mhc_chain'])
                    #,threshold=1)

                # Accumulate the filtered contacts
                if not contacts_TCR_MHC.empty:
                    all_contacts_TCR_MHC = pd.concat([all_contacts_TCR_MHC, contacts_TCR_MHC], ignore_index=True)

            except Exception as e:
                print(f"Error processing contacts for PDB ID {pdb_id}: {e}")
        else:
            print(f"Missing chains for PDB ID: {pdb_id}. Skipping...")

    # Step 5: Check if we have valid data to calculate potentials
    if not all_contacts_TCR_MHC.empty:
        print("Calculating TCR-MHC potential")
        data_TCR_MHC = calculate_potential(all_contacts_TCR_MHC, peptide=True)
        output_p = "./model/TCR_MHC_weighted.csv"  # Output model path
        
        # Ensure the output directory exists
        os.makedirs(os.path.dirname(output_p), exist_ok=True)

        data_TCR_MHC.to_csv(output_p, index=False)
        print(f"TCR-MHC potential saved to {output_p}")
    else:
        print("No valid TCR-MHC contacts for the training set.")

if __name__ == "__main__":
    main()

32 contact files found in the folder.
Processing contacts for PDB ID: 7n2p
Processing contacts for PDB ID: 8rlv
Processing contacts for PDB ID: 7l1d
Processing contacts for PDB ID: 5ivx
Processing contacts for PDB ID: 1mwa
Processing contacts for PDB ID: 7pb2
Processing contacts for PDB ID: 8shi
Processing contacts for PDB ID: 7n2q
Processing contacts for PDB ID: 3gsn
Processing contacts for PDB ID: 2ol3
Processing contacts for PDB ID: 5w1v
Processing contacts for PDB ID: 6uon
Processing contacts for PDB ID: 5jzi
Processing contacts for PDB ID: 6mtm
Processing contacts for PDB ID: 7nme
Processing contacts for PDB ID: 8enh
Processing contacts for PDB ID: 7dzm
Processing contacts for PDB ID: 3ffc
Processing contacts for PDB ID: 4prh
Processing contacts for PDB ID: 7r80
Processing contacts for PDB ID: 1g6r
Processing contacts for PDB ID: 7ndu
Processing contacts for PDB ID: 8f5a
Processing contacts for PDB ID: 7na5
Processing contacts for PDB ID: 7dzn
Processing contacts for PDB ID: 5brz
